# functools

> The [functools](https://docs.python.org/3.8/library/functools.html) module is for higher-order functions: functions that act on or return other functions. In general, any callable object can be treated as a function for the purposes of this module.

***

## functools.cached_property

`cached_property` is applied to a method of a class that is normally expensive to compute but **also doesn't change throughout the lifetime of the object**. The function turns a method into a property and caches the results the first time the method is called. Every subsequent time the method is called the cached results are returned.

In [1]:
from functools import cached_property

In [2]:
class LongList:
    
    def __init__(self, data):
        self._data = data
        
    @cached_property
    def maximum(self):
        print('Only prints first time')
        return max(self._data)
    
    @cached_property
    def minimum(self):
        print('Only prints first time')
        return min(self._data)

In [3]:
import random

ls = LongList([random.randint(5, 500) for _ in range(1_000_000)])

print(ls.maximum, ls.minimum, sep='\t', end='\n\n')
print(ls.maximum, ls.minimum, sep='\t')

# Even when you change the underlying data structure
# that minimum and maximum work on
ls._data = [random.randint(100, 300) for _ in range(1_000_000)]
# minimum and maximum don't recompute 
print(ls.maximum, ls.minimum, sep='\t')

Only prints first time
Only prints first time
500	5

500	5
500	5


## functools.cmp_to_key

Transforms a comparison function into a [key function](https://docs.python.org/3.8/glossary.html#term-key-function). \
A comparison function is one that takes in two arguments, call them `a` and `b`, and returns -1 if a < b, 0 if a == b, or 1 if a > b.

In [4]:
from functools import cmp_to_key

In [5]:
@cmp_to_key
def compare(a, b):
    if a < b:
        return -1
    elif a > b:
        return 1
    return 0

In [6]:
ls = [random.randint(1, 50) for _ in range(50)]

print(ls)
print()

print(sorted(ls, key=compare))

[42, 23, 10, 16, 14, 7, 36, 40, 18, 36, 13, 45, 27, 20, 21, 35, 12, 34, 28, 39, 20, 22, 48, 3, 25, 43, 49, 46, 10, 46, 1, 39, 5, 13, 24, 25, 43, 23, 13, 49, 9, 14, 17, 43, 36, 22, 34, 47, 24, 42]

[1, 3, 5, 7, 9, 10, 10, 12, 13, 13, 13, 14, 14, 16, 17, 18, 20, 20, 21, 22, 22, 23, 23, 24, 24, 25, 25, 27, 28, 34, 34, 35, 36, 36, 36, 39, 39, 40, 42, 42, 43, 43, 43, 45, 46, 46, 47, 48, 49, 49]


## lru_cache

Saves the `maxsize` most recent calls of a function for memoization. It is used to save time and computation on function calls with the same arguments multiple times.

We'll use the fibonacci sequence to illustrate the performance boost.

In [16]:
from functools import lru_cache

In [20]:
def fib(n):
    """Return the nth fibonacci number"""
    if n <= 1:
        return 1
    return fib(n-1) + fib(n-2)

In [21]:
%time fib(36)

CPU times: user 6.02 s, sys: 25.2 ms, total: 6.04 s
Wall time: 6.36 s


24157817

In [23]:
@lru_cache
def fib(n):
    """Return the nth fibonacci number"""
    if n <= 1:
        return 1
    return fib(n-1) + fib(n-2)

In [24]:
%time fib(36)

CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 34.1 µs


24157817

## total_ordering

If a user defined class implements operations such as object1 > object2, known as rich comparison ordering methods then as long as you implement `__eq__` along with one of `__le__`, `__ge__`, `__lt__`, `__gt__`.